In [1]:
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm


In [2]:
# load in the data
df = pd.read_csv('MovieLens_20M_Dataset/edited_rating.csv')

N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

# split into train and test
df = shuffle(df)
cutoff = int(0.008*len(df))
cutoffe = int(0.01*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:cutoffe]

# initialize variables
K = 10 # latent dimensionality
mu = df_train.rating.mean()
epochs = 15
bs = 1
lr = 0.01
reg = 0. # regularization penalty

In [3]:
N,M,K,len(df_train),len(df), mu

(138493, 26744, 10, 160002, 20000263, 3.526090298871264)

In [4]:
df_train.iloc[0:5]

,userId,movieId,rating,movie_idx
7815138,53907,4643,3.0,4551
5716903,39345,485,2.0,483
17664671,122163,2193,3.0,2111
389781,2658,1223,5.0,1199
5855940,40287,58998,4.0,12640


In [5]:
users = torch.tensor(df_train.userId.values)
items = torch.tensor(df_train.movie_idx.values)
ratings = torch.tensor(df_train.rating.values)

In [6]:
users, items, ratings

(tensor([ 53907,  39345, 122163,  ...,  75809,  83183,  69675]),
 tensor([4551,  483, 2111,  ..., 2342, 1081, 2678]),
 tensor([3.0000, 2.0000, 3.0000,  ..., 4.0000, 2.5000, 4.0000],
        dtype=torch.float64))

In [7]:
# test = torch.nn.Embedding(3,2)
# a = torch.tensor([0,1,2])
# test(a), a

In [8]:
class UserItemEmbeddingNet(torch.nn.Module):
    def __init__(self, n_users, n_items, k_factors, embedding_dropout = 0.02, hidden = 10, dropouts = 0.2):

        super(UserItemEmbeddingNet, self).__init__()
        self.u = torch.nn.Embedding(n_users, k_factors)
        self.m = torch.nn.Embedding(n_items, k_factors)
        self.u_bias = torch.nn.Embedding(n_users, 1)
        self.m_bias = torch.nn.Embedding(n_items, 1)

    def forward(self, users, items):
        uembed = self.u(users)
        membed = self.m(items)
        ubias = self.u_bias(users)
        mbias = self.m_bias(items)
        umdot = torch.mul(uembed,membed)
        umdot = torch.sum(umdot,1)
        umdot = torch.reshape(umdot, (umdot.shape[0], 1))
        output = torch.add(umdot, ubias)
        output = torch.add(output, mbias)
        output = output - mu
        output = torch.flatten(output)
        return output

UserItemEmbeddingNet = UserItemEmbeddingNet(N, M, K)
print(UserItemEmbeddingNet)

UserItemEmbeddingNet(
  (u): Embedding(138493, 10)
  (m): Embedding(26744, 10)
  (u_bias): Embedding(138493, 1)
  (m_bias): Embedding(26744, 1)
)


In [9]:
a = torch.tensor([[1],[2],[3]])
b = torch.tensor([1,3,2])
c = torch.add(a,b)
c, a.shape[0], b.shape[0]

(tensor([[2, 4, 3],
         [3, 5, 4],
         [4, 6, 5]]),
 3,
 3)

In [10]:
def fit():
    for epoch in range(epochs):
        print("Training epoch: ")
        for i in tqdm(range((N - 1) // bs + 1)):
            start_i = i * bs
            end_i = start_i + bs
            pred = UserItemEmbeddingNet(users, items)
            loss_func = torch.nn.MSELoss()
            loss = loss_func(pred.float(), ratings.float())

            loss.backward()
            with torch.no_grad():
                for p in UserItemEmbeddingNet.parameters():
                    p -= p.grad * lr
                UserItemEmbeddingNet.zero_grad()

In [ ]:
fit()

Training epoch: 


  3%|▎         | 4723/138493 [1:08:06<34:18:34,  1.08it/s]